# Analysis Notebook - Hierarchical Bayesian Modelling

## **NOTE**:

We assume that you have cloned the analysis repository and have `cd` into the parent directory. Before starting with the analysis make sure you have first completed the dependencies set up by following the instructions described in the **`dependencies/README.md`** document. All paths defined in this Notebook are relative to the parent directory (repository). Please close this Notebook and start again by following the above guidelines if you have not completed the aforementioned steps.

## Prerequisite input files

Before starting the execution of the following code, make sure you have available in the folders `sbas/data` and `sbas/assets` the files listed below as prerequisites.

###  **`sbas/data`**.
The present analysis requires the following files to be present in the folder **`sbas/data`**.


- [x] The contents of `data.tar.gz` after unpacking them into the `sbas/data` folder with `tar xvzf data.tar.gz -C sbas/data `
- [x] `SraRunTable.noCram.noExome.noWGS.totalRNA.txt`
- [x] `rmats_final.se.jc.ijc.txt`
- [x] `rmats_final.se.jc.sjc.txt`
- [x] `SraRunTable.noCram.noExome.noWGS.totalRNA.txt`


Additionally, the file `GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct` which is retrieved in the script from [`https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/`](https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz) and stored into the folder 
`sbas/data` as well.


### **`sbas/assets`**
The present analysis requires the following files to be present in the folder **`sbas/assets`**.

- [x] `tissues.tsv`: metadata file with information on which tissues will be used for analysis
- [x] `splice-relevant-genes.txt`: list of RNA binding proteins that are annotated to splicing relevant functions from GO.

## Loading dependencies

If `conda` is available on your environment you can install the required dependencies by running the following commands:


```bash
time conda install -y r-base==3.6.2 &&
conda install -y r-ggplot2 r-ggsci r-coda r-rstan r-rjags r-compute.es &&
Rscript -e 'install.packages("runjags", repos = "https://cloud.r-project.org/")'
```



In [ ]:
# dataviz dependencies
library(ggplot2)
library(ggsci)
library(grid)
library(gridExtra)

# BDA2E-utilities dependencies
library(parallel)
library(rjags)
library(runjags)
library(compute.es)

Download GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct from Google Cloud


In [24]:
if (!("GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct" %in% list.files("../data/"))) {
    message("Downloading GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct \nfrom https://console.cloud.google.com/storage/browser/_details/gtex_analysis_v7/rna_seq_data/ ..")
    system("wget -O ../data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz", intern = TRUE)
    message("Done!\n\n")
    message("Unzipping compressed file GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz..")
    system("gunzip ../data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz", intern = TRUE)
    message("Done! \n\nThe file GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct can be found in ../data/")
}

Previously used list of tissues to use for the Hierarchical Bayesian modelling:



```R
tissue.list<-c("Heart - Left Ventricle",
               "Breast - Mammary Tissue",
               "Brain - Cortex.Brain - Frontal Cortex (BA9).Brain - Anterior cingulate cortex (BA24)",
               "Adrenal Gland",
               "Adipose - Subcutaneous",
               "Muscle - Skeletal",
               "Thyroid",
               "Cells - Transformed fibroblasts",
               "Artery - Aorta",
               "Skin - Sun Exposed (Lower leg).Skin - Not Sun Exposed (Suprapubic)")
```

In [25]:
tissues_df <- readr::read_delim("../assets/tissues.tsv", delim = "\t")

Parsed with column specification:
cols(
  name = col_character(),
  female = col_double(),
  male = col_double(),
  include = col_double(),
  display.name = col_character()
)



In [26]:
tissue.list <- tissues_df$name[ tissues_df$include ==1]

In [28]:
cat(tissue.list, sep = ", ")

adipose_subcutaneous, adipose_visceral_omentum, adrenal_gland, artery_aorta, artery_coronary, artery_tibial, brain_caudate_basal_ganglia, brain_cerebellar_hemisphere, brain_cerebellum, brain_cortex, brain_frontal_cortex_ba_9, brain_hippocampus, brain_hypothalamus, brain_nucleus_accumbens_basal_ganglia, brain_putamen_basal_ganglia, brain_spinal_cord_cervical_c_1, breast_mammary_tissue, cells_cultured_fibroblasts, cells_ebv_transformed_lymphocytes, colon_sigmoid, colon_transverse, esophagus_gastroesophageal_junction, esophagus_mucosa, esophagus_muscularis, heart_atrial_appendage, heart_left_ventricle, liver, lung, muscle_skeletal, nerve_tibial, pancreas, pituitary, skin_not_sun_exposed_suprapubic, skin_sun_exposed_lower_leg, small_intestine_terminal_ileum, spleen, stomach, thyroid, whole_blood

In [30]:
tissue <- tissue.list[1]  #can be replaced with a loop or argument to choose a different tissue

In [31]:
tissue

[1] "adipose_subcutaneous"

In [8]:
dataDir <- "../data/"
assetsDir <- "../assets/"

In [9]:
ensg_map_path  <- paste0(dataDir, tissue, "_DGE_ensg_map.csv" ) 
DGE_file_path  <- paste0(dataDir, tissue, "_DGE.csv" ) 

ensg_map <- read.csv(ensg_map_path)
dge_file <- read.csv(DGE_file_path)

# Trim gene version after dot
dge_file[["ensg_names"]] <- gsub("\\..*","",rownames(dge_file))
events.table  <- dplyr::left_join(dge_file, ensg_map , by = "ensg_names")
rownames(events.table) <- events.table[["ensg_names"]]

Warning message:
“Column `ensg_names` joining character vector and factor, coercing into character vector”


In [10]:
head(ensg_map, 2)
head(dge_file, 2)
head(events.table, 2)

,ensg_names,ensg_genes
,<fct>,<fct>
1,ENSG00000176728,TTTY14
2,ENSG00000260197,AC010889.1


,logFC,AveExpr,t,P.Value,adj.P.Val,B,ensg_names
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
ENSG00000176728.7,-7.748207,-1.0323929,-104.4253,4.803353e-308,2.680943e-303,671.1671,ENSG00000176728
ENSG00000260197.1,-8.474073,-0.7043022,-101.6645,3.159370e-303,8.816855e-299,663.9100,ENSG00000260197


,logFC,AveExpr,t,P.Value,adj.P.Val,B,ensg_names,ensg_genes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
ENSG00000176728,-7.748207,-1.0323929,-104.4253,4.803353e-308,2.680943e-303,671.1671,ENSG00000176728,TTTY14
ENSG00000260197,-8.474073,-0.7043022,-101.6645,3.159370e-303,8.816855e-299,663.9100,ENSG00000260197,AC010889.1


In [11]:
events.table.name     <- paste0(dataDir, "fromGTF.SE.txt")

In [12]:
inc.counts.file.name  <- paste0(dataDir, "rmats_final.se.jc.ijc.txt")

In [13]:
skip.counts.file.name <- paste0(dataDir, "rmats_final.se.jc.sjc.txt")

In [14]:
rbp.table.name        <- paste0(assetsDir, "splice-relevant-genes.txt")

In [15]:
metadata.file.name    <- paste0(dataDir, "SraRunTable.noCram.noExome.noWGS.totalRNA.txt")

In [16]:
expression.file.name  <- paste0(dataDir, "GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct")

Load the skip and inclusion count matrices, and the list of RNA binding proteins that are annotated to either:
- mRNA splicing, via spliceosome `(GO:0000398)`,
- regulation of mRNA splicing, via spliceosome `(GO:0048024)`, or 
- both. 

The table has the:
- Gene Symbol
- the Uniprot ID (`uprot.id`)
- the NCBI Gene ID (`gene.id`) and 
- boolean columns for being 
  - `S`=mRNA splicing, via spliceosome `(GO:0000398)` and 
  - `R`=regulation of mRNA splicing, via spliceosome `(GO:0048024)`.

In [17]:
events.table <- events.table[abs(events.table$logFC)>=log2(1.5) & events.table$adj.P.Val<=0.05,]

In [18]:
annot.table  <- read.table(events.table.name,header=T)

In [19]:
rownames(annot.table) <- annot.table$ID

In [20]:
merged.table <- merge(events.table,annot.table,by="row.names")

In [21]:
rbp.table    <- read.table(rbp.table.name,sep="\t",header=TRUE)  

In [22]:
inc.counts   <- read.csv(inc.counts.file.name)

In [ ]:
skip.counts  <- read.csv(skip.counts.file.name)

## Check `dim()` of loaded objects

In [ ]:
message("Discrepancy in current with previous format, `ID` column not present")

In [ ]:
dim(events.table)
dim(annot.table)
dim(merged.table)
dim(rbp.table)
dim(inc.counts)
dim(skip.counts)

## Read sample info

In [ ]:
meta.data    <- read.csv(metadata.file.name,header=TRUE)
meta.data    <- meta.data[meta.data$body_site==tissue,]
inc.counts   <- inc.counts[,colnames(inc.counts) %in% meta.data$Run]
skip.counts  <- skip.counts[,colnames(skip.counts) %in% meta.data$Run]
sd.threshold <- quantile(apply(inc.counts,1,sd)+apply(skip.counts,1,sd),0.95)
skip.counts  <- skip.counts[rownames(skip.counts) %in% merged.table$Row.names,]
inc.counts   <- inc.counts[rownames(inc.counts) %in% merged.table$Row.names,]

if (nrow(skip.counts)>100)
{
  select.events<-apply(inc.counts,1,sd)+apply(skip.counts,1,sd)>sd.threshold
  inc.counts<-inc.counts[select.events,]
  skip.counts<-skip.counts[select.events,]
  merged.table<-merged.table[select.events,]
}

## Read expression data:

In [ ]:
expression.mat          <- read.table(expression.file.name, nrows=1,sep="\t",header=T,skip=2)
colnames.expression.mat <- colnames(expression.mat)
total.samples           <- length(colnames.expression.mat)
meta.data$Sample.Name   <- gsub("-","\\.",meta.data$Sample.Name)
meta.data               <- meta.data[meta.data$Sample.Name %in% colnames(expression.mat),]
meta.data               <- meta.data[!duplicated(meta.data$Sample.Name),]
inc.counts              <- inc.counts[,colnames(inc.counts) %in% meta.data$Run]
skip.counts             <- skip.counts[,colnames(skip.counts) %in% meta.data$Run]
meta.data               <- meta.data[meta.data$Run %in% colnames(inc.counts),]
col.in.tissue           <- c()

for (col in colnames.expression.mat)
  col.in.tissue<-c(col.in.tissue, (col %in% meta.data$Sample.Name) && (meta.data$body_site[which(meta.data$Sample.Name==col)] %in% tissue) && (meta.data$submitted_subject_id[which(meta.data$Sample.Name==col)]!="GTEX-11ILO"))
expression.mat<-read.table(expression.file.name, colClasses = ifelse(col.in.tissue,"numeric","NULL"),sep="\t",header=T,skip=2)

## Read gene names:

In [ ]:
expression.mat <- expression.mat[,order(match(colnames(expression.mat),meta.data$Sample.Name))]
inc.counts     <- inc.counts[,order(match(colnames(inc.counts),meta.data$Run))]
skip.counts    <- skip.counts[,order(match(colnames(skip.counts),meta.data$Run))]
all.genes      <- read.table(expression.file.name,sep="\t",header=T,skip=2,colClasses = c(rep("character", 2), rep("NULL", total.samples-2)))
expression.mat <- expression.mat[!duplicated(all.genes$Description),]
all.genes      <- all.genes[!duplicated(all.genes$Description),]
skip.counts    <- skip.counts[merged.table$geneSymbol %in% all.genes$Description,]
inc.counts     <- inc.counts[merged.table$geneSymbol %in% all.genes$Description,]
merged.table   <- merged.table[merged.table$geneSymbol %in% all.genes$Description,]
gene.names     <- unique(merged.table$geneSymbol)
expression.mat <- expression.mat[all.genes$Description %in% c(as.character(rbp.table$Gene),as.character(gene.names)),]
rownames.expression.mat <-all.genes$Description[all.genes$Description %in% c(as.character(rbp.table$Gene),as.character(gene.names))]
expression.mat <-expression.mat[!duplicated(rownames.expression.mat),]
rownames.expression.mat <-rownames.expression.mat[!duplicated(rownames.expression.mat)]

## Prepare expression of genes and RBPS:

In [ ]:
num.events     <- nrow(merged.table)
event.to.gene  <- c()
gexp           <- expression.mat[rownames.expression.mat %in% gene.names,]
rownames(gexp) <- rownames.expression.mat[rownames.expression.mat %in% gene.names]
gexp           <- gexp[order(match(rownames(gexp),gene.names)),]
gexp           <- log2(gexp+0.5)
gexp           <- gexp-rowMeans(gexp)

gexp[apply(gexp,1,sd)>0,] <- gexp[apply(gexp,1,sd)>0,]/apply(gexp[apply(gexp,1,sd)>0,],1,sd)
rexp           <- expression.mat[rownames.expression.mat %in% rbp.table$Gene,]
rownames(rexp) <- rownames.expression.mat[rownames.expression.mat %in% rbp.table$Gene]
rexp           <- rexp[order(match(rownames(rexp),rbp.table$Gene)),]
rexp           <- log2(rexp+0.5)
rexp           <- rexp-rowMeans(rexp)
rexp           <- rexp/apply(rexp,1,function(v){ifelse(sum(v==v[1])<length(v),sd(v),1)})

for (i in (1:num.events))
  event.to.gene<-c(event.to.gene,which(unique(merged.table$geneSymbol)==merged.table[i,"geneSymbol"]))
sex<-ifelse(meta.data$sex=="male",1,0)


## Run stan:

In [ ]:
dataList = list(
  as = round(skip.counts) ,   #skip event counts across experiments
  c = round(skip.counts+inc.counts)    , #total counts for event, i.e. skip+inclusion, across experiments
  gexp = gexp, #read counts for genes (from gtex, take the raw counts) across experiments
  rexp = rexp, #read counts for RBPs (from gtex, take the raw counts)
  event_to_gene = event.to.gene,  #the gene index for each event (1 to the number of distinct genes) 
  Nrbp = nrow(rexp), #number of RBPs
  Nevents = nrow(merged.table),  #most varying AS events in 
  Nexp = ncol(expression.mat),#number of experiments such that we measured each event, gene and RBP in each experiment
  Ngenes = nrow(gexp),
  sex=sex
)


modelString = "
data {
int<lower=0> Nevents;
int<lower=0> Nexp;
int<lower=0> Nrbp;
int<lower=0> Ngenes;
int<lower=0> as[Nevents,Nexp] ;
int<lower=0> c[Nevents,Nexp] ;
matrix[Ngenes,Nexp] gexp ; 
matrix[Nrbp,Nexp] rexp ; 
int<lower=0> event_to_gene[Nevents];
int<lower=0,upper=1> sex[Nexp];

}


parameters {
real beta0[Nevents] ;
real beta1[Nevents] ;
matrix[Nevents,Nrbp] beta2 ;
real beta3[Nevents];
real beta4[Nrbp];

}
model {

for ( i in 1:Nexp ) {  


    for ( j in 1:Nevents ) if (c[j,i]>0) { 

      as[j,i] ~ binomial(c[j,i], inv_logit(beta0[j]+beta1[j]*sex[i]+dot_product(beta2[j,],rexp[,i])+beta3[j]*gexp[event_to_gene[j],i] ) );

  }
}

for (k in 1:Nrbp){

  for ( j in 1:Nevents ) { 

        beta2[j,k] ~normal(beta4[k],1);
  }

  beta4[k]~normal(0,1);

}


for ( j in 1:Nevents ) { 

    beta1[j] ~ normal(0,1);
    beta0[j] ~ normal(0,1);
    beta3[j] ~ normal(0,1);
  }

}
"

stanDso <- stan_model( model_code=modelString ) 
stanFit <- sampling( object=stanDso , data = dataList , chains = 3 ,iter = 8000,warmup=6000   , thin = 1,init=0, cores=3 )
mcmcCoda = mcmc.list( lapply( 1:ncol(stanFit) , function(x) { mcmc(as.array(stanFit)[,x,]) } ) )

# > Inspect from here onwards

- What files are needed from here to be saved?
- Is the diagnostic plotting helpful for keeping in this notebook?

In [ ]:
source("../dimorphAS/DBDA2Eprograms/DBDA2E-utilities.R")

## Initialising dataframe with columns `coef,rbp,tissue`

In [ ]:
df <-data.frame(coef=NULL,rbp=NULL,tissue=NULL)

Before running the following, use the Session menu to set working directory to source file location
```R
setwd(dir = "../dimorphAS/DBDA2Eprograms/")
```

In [ ]:
load("../dimorphAS/figures/oldFigureDrafts/figure3b.RData")

### This invokes X11 which is not available for all systems, and won"t work in a NextFlow pipeline

```
diagMCMC(mcmcCoda , parName=c("beta2[101,87]"))
```

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

codaObject <- mcmcCoda 
parName    <- c("beta2[101,87]") #varnames(codaObject)[1]
saveName   <- NULL
saveType   <- "jpg"


DBDAplColors = c("skyblue",
               "black",
               "royalblue",
               "steelblue")

#openGraph(height=5,width=7)
    
par(mar=0.5+c(3,4,1,0) , 
  oma=0.1+c(0,0,2,0) , 
  mgp=c(2.25,0.7,0) , 
  cex.lab=1.5 )
    
layout(matrix(1:4,nrow=2))
  # traceplot and gelman.plot are from CODA package:
require(coda)
coda::traceplot( codaObject[,c(parName)], 
              main="" , 
              ylab="Param. Value" ,
              col=DBDAplColors )

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)
tryVal = try(
coda::gelman.plot(codaObject[,c(parName)] , 
                  main="",
                  auto.layout=FALSE,
                  col=DBDAplColors )
)  

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

# if it runs, gelman.plot returns a list with finite shrink values:
  if ( class(tryVal)=="try-error" ) {
    plot.new() 
    print(paste0("Warning: coda::gelman.plot fails for ",parName))
  } else { 
    if ( class(tryVal)=="list" & !is.finite(tryVal$shrink[1]) ) {
      plot.new() 
      print(paste0("Warning: coda::gelman.plot fails for ",parName))
    }
  }
  DbdaAcfPlot(codaObject,parName,plColors=DBDAplColors)
  DbdaDensPlot(codaObject,parName,plColors=DBDAplColors)
  mtext( text=parName , outer=TRUE , adj=c(0.5,0.5) , cex=2.0 )
  if ( !is.null(saveName) ) {
    saveGraph( file=paste0(saveName,"Diag",parName), type=saveType)
  }



In [ ]:
##Collect coefficients for RBPs whose 95% HDI does not contain 0:

In [ ]:
rbp.names<-rownames(rexp)

df<-data.frame(coef=NULL,rbp=NULL,tissue=NULL)

hdi<-HPDinterval(mcmcCoda)  

s <- summary(mcmcCoda)

m <- s$statistics[,"Mean"]

beta2.mat<-matrix(nrow=nrow(merged.table),ncol=length(rbp.names))

for (rbp in (1:length(rbp.names)))
  
  for (event in (1:nrow(merged.table)))
  {
    
    var.name<-paste0("beta2[",event,",",rbp,"]")
    
    low<-hdi[[1]][rownames(hdi[[1]])==var.name][1]
    
    high<-hdi[[1]][rownames(hdi[[1]])==var.name][2]
    
    beta2.mat[event,rbp]<-m[grepl(paste0("beta2\\[",event,",",rbp,"\\]"),names(m))]
    
    if (low<0 && high>0)
      
      beta2.mat[event,rbp]<-0
    
  }



colnames(beta2.mat)=rbp.names

for (rbp in rbp.names)
  
  df<-rbind(df,cbind(beta2.mat[,colnames(beta2.mat)==rbp],rep(rbp,nrow(beta2.mat)),rep(tissue,nrow(beta2.mat)))  )


colnames(df)<-c("Coef","RBP","Tissue")
    
df$Coef<-as.numeric(as.character(df$Coef))


In [ ]:
##Display a violin plot for some selected RBPs:

In [ ]:
labels<-read.table("labels.tsv",sep="\t",header=T)

df$Tissue<-as.character(df$Tissue)

for (i in 1:nrow(df))
  
  if (df$Tissue[i] %in% labels$tissue)
    
    df$Tissue[i]<-as.character(labels$X[which(as.character(labels$tissue)==as.character(df$Tissue[i]))])


df<-df[df$Coef!=0,]


df$RBP<-as.character(df$RBP)

df1<-df[df$RBP %in% c("BUD13", "GTF2F1","CTNNBL1"),]

pn1<-ggplot(df1,aes(factor(RBP),Coef)) + geom_violin(aes(fill="red")) + scale_fill_manual(values = "#4DBBD5FF") 

pn1 <- pn1 + theme_minimal() +  theme(text = element_text(size=20),
                                      axis.text = element_text(size=20, hjust=0.5),
                                      axis.title.x=element_blank(),
                                      axis.title.y = element_text(size=24),
                                      plot.title = element_text(hjust = 0.5),
                                      legend.position = "none") + ylab("") + labs(title="")+ylim(-2,2)+ geom_hline(yintercept=0)



In [ ]:
##Create barplot of the number of RBPs that tend to promote skipping, the number of RBPs that tend to promote
#inclusion and the number of RBPs whose effect is context-specific, for the two RBP groups

In [ ]:
spliceosome_genes = as.character(rbp.table$Gene[rbp.table$S=="TRUE"])

splice_regulation_genes = as.character(rbp.table$Gene[rbp.table$R=="TRUE"])

for (RBP_set in list(spliceosome=spliceosome_genes,splice_regulation=splice_regulation_genes))
{
  sum.pos<-sort(unlist(lapply(lapply(split(df$Coef[df$RBP %in% RBP_set],df$RBP[df$RBP %in% RBP_set]),">",0),sum)),decreasing = T)
  
  sum.neg<-sort(unlist(lapply(lapply(split(df$Coef[df$RBP %in% RBP_set],df$RBP[df$RBP %in% RBP_set]),"<",0),sum)),decreasing = T)
  
  sum.pos<-sum.pos[order(names(sum.pos))]
  
  sum.neg<-sum.neg[order(names(sum.neg))]
  
  pos.rbps<-names(which(sum.pos/(sum.pos+sum.neg)>=0.75 & (sum.pos+sum.neg>quantile(sum.pos+sum.neg,0.2))))
  
  neg.rbps<-names(which(sum.pos/(sum.pos+sum.neg)<=0.25 & (sum.pos+sum.neg>quantile(sum.pos+sum.neg,0.2))))
  
  cs.rbps<-names(which(sum.pos/(sum.pos+sum.neg)>0.25 & sum.pos/(sum.pos+sum.neg)<0.75 & (sum.pos+sum.neg>quantile(sum.pos+sum.neg,0.2))))
  
  df.counts<-data.frame(type=c("Skip","Inc","CS"),counts=c(length(pos.rbps),length(neg.rbps),length(cs.rbps)))
  
  pn4_new <- ggplot(df.counts, aes(type, counts)) +  
    geom_bar(fill = "#00008B",color="black", position = "dodge", stat="identity") + 
    geom_text(aes(x = type, y = counts + 10, label = paste(100 * round(counts/sum(counts), 3), "%", sep = "")), size = 3) +
    guides(fill=FALSE) +
    xlab("") + scale_y_continuous(breaks = c(0, 20, 40), limits = c(0, 60))+
    theme_minimal() +
    theme(
      axis.text = element_text(size = 8), 
      axis.text.x = element_text(angle = 90, hjust = 1), 
      axis.title = element_text(size = 10),
      axis.title.y = element_text(vjust = 5)
    )
  show(pn4_new  )
}




In [ ]:
if ( ("lv.txt" %in% list.files("../data/")) && ("mt.txt" %in% list.files("../data/"))) {
        message("The files lv.txt or mt.txt are available in the folder ../data/! \n")
        message("The "perl parseMT.pl" command will not be re-run \n")
}


if ( (!("lv.txt" %in% list.files("../data/"))) | (!("mt.txt" %in% list.files("../data/")))) {
        message("The files lv.txt or mt.txt not found in the folder ../data/ \n")
        message("Generating lv.txt and mt.txt with "perl parseMT.pl" using "summary_hbm.txt" as input .. \n")
        system(paste0("cd ../dimorphAS/notebook/ && ",
                      "perl parseMT.pl > parseMT_output.txt && ", 
                      "mv lv.txt ../../data/ && ",
                      "mv mt.txt ../../data/ && ",
                      "cp summary_hbm.txt  ../../data/"), 
               intern  = TRUE)
        message("Done!\n")
}



## Metadata

For replicability and reproducibility purposes, we also print the following metadata:

1. Checksums of **"artefacts"**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

### 1. Checksums with the sha256 algorithm

In [ ]:
figure_id       <- "figures_3"

message("Generating sha256 checksums of the artefacts in the `..data/` directory .. ")
system(paste0("cd ../data/ && find . -type f -exec sha256sum {} \\; > ../metadata/",  figure_id, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

data.table::fread(paste0("../metadata/", figure_id, "_sha256sums.txt"), header = FALSE, col.names = c("sha256sum", "file"))

### 2. Libraries metadata

In [ ]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", figure_id, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", figure_id ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]